# ***1. Install and Import Dependencies***

In [14]:
import numpy as np
import pandas as pd

In [2]:
!pip3 install torch torchvision torchaudio

In [3]:
! pip3 install transformers requests beautifulsoup4 pandas numpy

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# ***2. Instantiate Model***

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# ***3. Encode and Calculate Sentiment***

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')


In [7]:
result = model(tokens)

In [9]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [10]:
int(torch.argmax(result.logits))+1

4

# ***4. Collect Reviews***

In [11]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment*.')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [13]:
reviews[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

# ***5. Load Reviews into DataFrame and Score***

In [15]:
df = pd.DataFrame(np.array(reviews),columns =['review'])

In [17]:
df['review'].iloc[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [18]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [20]:
sentiment_score(df['review'].iloc[0])

4

In [21]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [22]:
df.head()

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,The food and service here was really good. It...,5
2,Hotter than I imagine myself to be and firing ...,5
3,Visiting from Texas and decided to give this r...,5
4,Don't come here expecting legit Mexican food b...,3
